In [1]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd
import os

### Retrieve games and team names using NBA API

In [2]:
nba_teams = teams.get_teams()
gamefinder = leaguegamefinder.LeagueGameFinder()
games = gamefinder.get_data_frames()[0]
games.head()

total_df = pd.DataFrame()
print(games.columns)

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS'],
      dtype='object')


In [3]:
team_ids = []
team_names = []
for team in nba_teams:
    team_names.append((team['id'], team['abbreviation']))

### Convert GAME_DATE column to date_time

In [4]:
games.GAME_DATE = pd.to_datetime(games.GAME_DATE)

print(games.GAME_DATE.min())
print(games.GAME_DATE.max())

2014-11-22 00:00:00
2021-04-25 00:00:00


### Filter games and find shooting stats against each team

In [5]:
for (id, name) in team_names:
    games_against_team = games[games.MATCHUP.str.contains(name)]
    games_against_team = games_against_team[games_against_team['TEAM_ABBREVIATION'] != name]
    average_against = games_against_team.mean(axis = 0)
    team_vals = pd.DataFrame()
    team_vals = average_against.filter(items = ['FG_PCT', 'FG3_PCT', 'FT_PCT'])
    team_vals['TEAM'] = name
    total_df = total_df.append(team_vals, ignore_index = True)

total_df

C:\Users\gourn\anaconda3\envs\dva\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  after removing the cwd from sys.path.


,FG3_PCT,FG_PCT,FT_PCT,TEAM
0,0.342481,0.446935,0.773910,ATL
1,0.335146,0.443535,0.761766,BOS
2,0.353561,0.465159,0.759452,CLE
3,0.354838,0.459733,0.771238,NOP
4,0.347175,0.451897,0.767129,CHI
5,0.353061,0.457709,0.768481,DAL
6,0.354221,0.463468,0.757494,DEN
7,0.342478,0.441065,0.762289,GSW
8,0.347212,0.460947,0.755608,HOU
9,0.346620,0.448515,0.758156,LAC


In [6]:
stats = total_df

### Append state to average stats DataFrame

In [7]:
datapath = os.getcwd()
states = pd.read_csv(datapath + '/state_teams.csv')
states.set_index('TEAM')

,State
TEAM,
ATL,Georgia
BOS,Massachusetts
CLE,Ohio
NOP,New Orleans
CHI,Illinois
DAL,Texas
DEN,Colorado
GSW,California
HOU,Texas


In [8]:
data = states.join(stats.set_index('TEAM'), on='TEAM')

data.set_index('TEAM')

,State,FG3_PCT,FG_PCT,FT_PCT
TEAM,,,,
ATL,Georgia,0.342481,0.446935,0.773910
BOS,Massachusetts,0.335146,0.443535,0.761766
CLE,Ohio,0.353561,0.465159,0.759452
NOP,New Orleans,0.354838,0.459733,0.771238
CHI,Illinois,0.347175,0.451897,0.767129
DAL,Texas,0.353061,0.457709,0.768481
DEN,Colorado,0.354221,0.463468,0.757494
GSW,California,0.342478,0.441065,0.762289
HOU,Texas,0.347212,0.460947,0.755608


In [9]:
data.to_csv('data.csv', index='False')

### Extract bubble games and find average stats in bubble

In [10]:
games_bubble = games[(games.GAME_DATE > '2020-07-01') & (games.SEASON_ID.str[-4:] == '2019')]

In [11]:
bubble_df = pd.DataFrame()
for (id, name) in team_names:
    games_against_team = games_bubble[games_bubble.MATCHUP.str.contains(name)]
    games_against_team = games_against_team[games_against_team['TEAM_ABBREVIATION'] != name]
    average_against = games_against_team.mean(axis = 0)
    team_vals = pd.DataFrame()
    team_vals = average_against.filter(items = ['FG_PCT', 'FG3_PCT', 'FT_PCT'])
    team_vals['TEAM'] = name
    bubble_df = bubble_df.append(team_vals, ignore_index = True)

C:\Users\gourn\anaconda3\envs\dva\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  """


In [12]:
bubble_df.dropna()
bubble_avgs = bubble_df.mean(axis=0)

bubble_avgs = bubble_avgs.reset_index()
bubble_avgs = bubble_avgs.rename(columns={0: 'value', 'index': 'stat'}).set_index('stat')

bubble_avgs

,value
stat,
FG3_PCT,0.360107
FG_PCT,0.459553
FT_PCT,0.786202


In [13]:
bubble_avgs.to_csv('bubble_data.csv', index='False')